In [1]:
pip install faker

In [2]:
import sqlite3
from faker import Faker
import random
from datetime import datetime

# Crear una instancia de Faker
fake = Faker()

# Conectar a la base de datos SQLite (se creará automáticamente)
conn = sqlite3.connect('tienda_minorista.db')
cursor = conn.cursor()

# Crear tabla de Ventas sin la columna Competitor_Price
cursor.execute('''CREATE TABLE IF NOT EXISTS Ventas (
                    Customer_Id INTEGER,
                    DateTime TEXT,
                    Customer_Name TEXT,
                    Customer_Category TEXT,
                    Marital_Status TEXT,
                    Age INTEGER,
                    Gender TEXT,
                    Products TEXT,
                    Total_Items INTEGER,
                    Total_Price REAL,
                    Payment_Method TEXT,
                    State TEXT,
                    Season TEXT,
                    PCL TEXT)''')

# Lista de categorías de clientes y rangos de edad
customer_categories = ['Student', 'Professional', 'Homemaker', 'Teenager', 'Young', 'Adult', 'Elderly']
age_ranges = {'Student': (14, 17), 'Professional': (18, 29), 'Homemaker': (30, 64), 'Teenager': (14, 17), 'Young': (18, 29), 'Adult': (30, 64), 'Elderly': (65, 100)}

# Probabilidades para las categorías de clientes
category_probabilities = [0.1, 0.2, 0.1, 0.1, 0.15, 0.2, 0.15]

# Probabilidades para el estado civil
marital_status_probabilities = [0.3, 0.3, 0.2, 0.2]

# Probabilidades para los métodos de pago
payment_method_probabilities = [0.2, 0.2, 0.1, 0.5]

# Lista de géneros
genders = ['Male', 'Female', 'Other']

# Lista de productos de comida en inglés
food_products = ['Rice', 'Beans', 'Chicken', 'Fish', 'Salad', 'Soup', 'Pasta', 'Burger', 'Pizza', 'Tacos',
                 'Steak', 'Sushi', 'Sandwich', 'Fries', 'Curry', 'Burrito', 'Lasagna', 'Hotdog', 'Ramen', 'Donut',
                 'Eggs', 'Milk', 'Bread', 'Cheese', 'Yogurt', 'Apple', 'Banana', 'Orange', 'Broccoli', 'Carrot',
                 'Tomato', 'Cucumber', 'Lettuce', 'Potato', 'Onion', 'Garlic', 'Sausage', 'Ham', 'Bacon', 'Salami']

# Precios de los productos
food_prices = {'Rice': 1.99, 'Beans': 2.5, 'Chicken': 5.99, 'Fish': 7.99, 'Salad': 4.5, 'Soup': 3.99, 'Pasta': 3.25,
               'Burger': 6.5, 'Pizza': 8.99, 'Tacos': 5.25, 'Steak': 12.99, 'Sushi': 9.99, 'Sandwich': 4.99,
               'Fries': 2.99, 'Curry': 6.75, 'Burrito': 7.25, 'Lasagna': 8.5, 'Hotdog': 3.5, 'Ramen': 4.25, 'Donut': 1.25,
               'Eggs': 2.99, 'Milk': 3.49, 'Bread': 2.25, 'Cheese': 4.99, 'Yogurt': 2.75, 'Apple': 0.75, 'Banana': 0.5,
               'Orange': 0.99, 'Broccoli': 1.99, 'Carrot': 0.75, 'Tomato': 1.25, 'Cucumber': 1.49, 'Lettuce': 1.75,
               'Potato': 1.25, 'Onion': 1.0, 'Garlic': 0.5, 'Sausage': 3.99, 'Ham': 4.5, 'Bacon': 5.99, 'Salami': 4.25}

# Lista de estados de USA
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
    'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
    'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota',
    'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
    ]

# Pesos para cada estado (proporciones aproximadas)
state_weights = [10, 20, 15, 10, 50, 20, 10, 5, 30, 25, 5, 25, 15, 10, 10, 10, 15, 5, 10, 10, 30, 10, 15, 15, 10, 5, 10, 10, 15, 20, 30, 100, 10, 5, 20, 10, 30, 10, 10, 5, 10, 10, 20, 50, 15, 5, 25, 10, 5, 5]

# Probabilidades para la asignación de estados a Customer_Id
state_assignment_probabilities = [0.85, 0.1, 0.05]

# Probabilidades para PCL
pcl_probabilities = [0.1, 0.9]

# Generar los registros
customer_id_states = {}  # Diccionario para rastrear los estados asignados a cada Customer_Id

# Diccionarios para rastrear los valores de Gender, Age, Marital_Status, Customer_Category y Customer_Name
customer_info = {}

for _ in range(200000):
    Customer_Id = random.randint(1000, 9999)
    DateTime = fake.date_time_between(start_date='-1y', end_date='now')

    # Obtener la temporada según la fecha
    if 3 <= DateTime.month <= 5:
        Season = 'Spring'
    elif 6 <= DateTime.month <= 8:
        Season = 'Summer'
    elif 9 <= DateTime.month <= 11:
        Season = 'Autumn'
    else:
        Season = 'Winter'

    DateTime = DateTime.strftime('%Y-%m-%d %H:%M:%S')

    # Generar o recuperar información del cliente
    if Customer_Id not in customer_info:
        Customer_Name = fake.first_name() + ' ' + fake.last_name()
        # Seleccionar aleatoriamente una categoría de cliente
        customer_category = random.choices(customer_categories, weights=category_probabilities)[0]

        # Generar estado civil para las categorías de cliente
        if customer_category in ['Student', 'Teenager']:
            Marital_Status = 'Single'
        elif customer_category == 'Young':
            Marital_Status = random.choices(['Single', 'Married'], weights=[0.5, 0.5])[0]
        else:
            Marital_Status = random.choices(['Single', 'Married', 'Divorced', 'Widowed'], weights=marital_status_probabilities)[0]

        # Generar una edad aleatoria dentro del rango correspondiente a la categoría de cliente
        age_range = age_ranges[customer_category]
        Age = random.randint(age_range[0], age_range[1])

        # Elegir el género aleatoriamente, con probabilidades ajustadas
        gender_probabilities = [0.35, 0.55, 0.1]  # Male, Female, Other
        Gender = random.choices(genders, weights=gender_probabilities)[0]

        # Generar valor para la columna PCL (Programa de Fidelización de Cliente)
        PCL = random.choices(['True', 'False'], weights=pcl_probabilities)[0]

        # Guardar la información del cliente en el diccionario customer_info
        customer_info[Customer_Id] = {
            'Customer_Name': Customer_Name,
            'Customer_Category': customer_category,
            'Marital_Status': Marital_Status,
            'Age': Age,
            'Gender': Gender,
            'PCL': PCL
        }
    else:
        # Recuperar la información del cliente del diccionario
        Customer_Name = customer_info[Customer_Id]['Customer_Name']
        customer_category = customer_info[Customer_Id]['Customer_Category']
        Marital_Status = customer_info[Customer_Id]['Marital_Status']
        Age = customer_info[Customer_Id]['Age']
        Gender = customer_info[Customer_Id]['Gender']
        PCL =  customer_info[Customer_Id]['PCL']

    # Generar una lista de productos de comida aleatorios en inglés con
    products = [random.choice(food_products) for _ in range(random.randint(1, 5))] # Generar entre 1 y 5 productos aleatorios
    total_items = len(products)  # Contar la cantidad de productos
    Total_Price = sum(food_prices[product] for product in products)  # Calcular el precio total de los productos

    # Seleccionar método de pago con probabilidades ajustadas
    Payment_Method = random.choices(['Credit Card', 'Debit Card', 'Cash', 'Mobile Payment'], weights=payment_method_probabilities)[0]

    # Asignar estado(es) al Customer_Id
    if Customer_Id not in customer_id_states:
        # Asignar un solo estado en un 85% de los casos
        # Dos estados diferentes en un 10% de los casos
        # Tres estados diferentes en un 5% de los casos
        num_states = random.choices([1, 2, 3], weights=state_assignment_probabilities)[0]
        assigned_states = random.choices(us_states, weights=state_weights, k=num_states)  # Se usa choices en lugar de sample
        customer_id_states[Customer_Id] = assigned_states
    else:
        assigned_states = customer_id_states[Customer_Id]

    # Tomar un estado aleatorio de los asignados al Customer_Id
    State = random.choice(assigned_states)

    cursor.execute('''INSERT INTO Ventas (Customer_Id, DateTime, Customer_Name, Customer_Category, Marital_Status, Age, Gender, Products, Total_Items, Total_Price, Payment_Method, State, Season, PCL)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                  (Customer_Id, DateTime, Customer_Name, customer_category, Marital_Status, Age, Gender, ', '.join(products), total_items, Total_Price, Payment_Method, State, Season, PCL))
# Guardar cambios y cerrar conexión
conn.commit()
conn.close()

print("Base de datos creada exitosamente con 200,000 registros en la tabla de Ventas.")


Base de datos creada exitosamente con 200,000 registros en la tabla de Ventas.


In [3]:
import sqlite3
import pandas as pd

# Conectar a la base de datos SQLite
conn = sqlite3.connect('tienda_minorista.db')

# Consultar la tabla de Ventas y cargar los datos en un DataFrame de pandas
df = pd.read_sql_query("SELECT * FROM Ventas", conn)

# Guardar el DataFrame en un archivo CSV
df.to_csv('DM_Sales.csv', index=False)

# Cerrar la conexión a la base de datos
conn.close()

print("Archivo CSV generado exitosamente.")

Archivo CSV generado exitosamente.
